About the number of students per class:
Not all classes need to have the same number of students. Classes without enough interest should not run. Students who put these as their top choice should then be punted to a lower level choice class.
 * The minimum number of students needed for a class to run and the maximum possible per class will be provided. It's best to let these be input variables to your function so that it can be adjusted.


About assigning students to classes: 

You should maximize overall happiness, where happiness is defined as  getting a higher ranked choice. 

If more students than a class' capacity rank it similarly, put students in the class randomly and the remaining students should be enrolled in their next highest choice. 

What is tricky is: you are trying to maximize student choices in aggregate (i.e., minimize the sum of all the student choices (#1 being the top choice, but the least number) while also preventing individual students from getting really low choice classes. 

If a students' top 6 choices are not among a class that runs, flag it as an issue that needs to be handled manually.

Dynamic Programming: https://www.youtube.com/watch?v=cJ21moQpofY


Linear Programming: https://www.youtube.com/watch?v=TYC2XMmFKWE

Forum: https://ask.metafilter.com/291896/How-do-I-sort-folks-into-groups-based-on-their-preferences#4228186

4 choices for classes

10 students per class, min 5 students

In [1]:
import random
import pandas as pd
import numpy as np

In [4]:
for x in range(1):
    print(random.choices([1,2,2],weights=[1,100,100]))

[2]


In [17]:
def get_choices(p):
    choices = []
    vals = {1:10,2:3,3:4,4:5,6:2,7:11,8:11,9:11,10:13,11:3,12:4,13:3,14:3,15:2}
    for x in range(5):
        weights = list(vals.values())
        options = list(vals.keys())
        choice = random.choices(options,weights=weights)[0]
        vals.pop(choice)
        choices.append(choice)
    return choices

people = [[x]+get_choices(x) for x in range(100)]
df = pd.DataFrame(people)
df.columns = ["Your name","1st","2nd","3rd","4th","5th"]
df.to_csv("../input/df2.csv",index=False)

In [112]:
df = pd.read_csv("../input/df.csv",index_col=False)
df.set_index("Your name")

1st  2nd  3rd  4th  5th
Your name                         
0            8    1    3   12   10
1            8   10   12   13    1
2           10    1    7    3    4
3            8    3    1    9   10
4            4    9    3    7    8
...        ...  ...  ...  ...  ...
95           1   11   10    7   13
96          14    2   11    1    4
97           7   10    9   15    4
98           9   14   10    7    2
99           7   13    4    9   14

[100 rows x 5 columns]

In [18]:
class FunctionTools:
    def getChoices(self,person):
        return list(self.df.loc[person])
    def first_choice(self):
        self.groups = {group:[]for group in self.allGroups}
        for person,choices in self.df.iterrows():
            self.groups[choices[0]] = self.groups.get(choices[0],[])+[[person,1]]
        
    def lengths(self):
        return [len(x) for x in self.groups.values()]
    def overCapacity(self):
        return [groupName for groupName,people in self.groups.items() if len(people)>self.maxCapacity]
    def getRank(self,choices, rank):
        return choices[rank-1]
    def removePersonFromGroup(self,person,groupName):
        self.groups[groupName] = [[person2,rank] for person2,rank in self.groups[groupName] if person2 != person] 
    def addPersonToGroup(self,groupName, person, rank):
        self.groups[groupName] = self.groups.get(groupName,[])+[[person,rank]]
    def getGroup(self,groupName):
        return self.groups[groupName]
    def getLengthGroupName(self,groupName):
        return len(self.groups[groupName])
    def Print(self):
        print("Eliminated Groups %s" % self.eliminatedGroups)
        for groupName, group in self.groups.items():
            print("GroupName:",groupName,"# of people",len(group))
        print("\n\n")
    def getPersonsRank(self,person,rank):
        choices = self.getChoices(person)
        newChoice = self.getRank(choices,rank)
        return newChoice
    def removePerson(self,person):
        for group in self.groups:
            self.removePersonFromGroup(person,group)
class Scheduling(FunctionTools):
    df = pd.read_csv("../input/df2.csv",index_col=False)
    df = df.set_index("Your name")
    maxCapacity = 12
    minCapacity = 6
    def __init__(self):
        self.eliminatedGroups = []
        self.allGroups = list(range(1,16))
        self.flagged= []

        #go to first choice
        self.first_choice()
        newEliminatedGroup = True
        
        while newEliminatedGroup != None:
            self.AdjustUnderCapacity()

            self.Adjust()
            self.Print()

            newEliminatedGroup = self.NewEliminatedGroup() #key eliminate one by one
            self.eliminatedGroups = self.eliminatedGroups + [newEliminatedGroup]
        
        
        self.Evaluate()
        print(self.flagged)
    
    def Adjust(self):
        #go to second choice
        self.AdjustGroups(2) #move overcapcity to second choice
        self.AdjustGroups(3) #to third
        self.ComplexAdjustGroups(2) #2+2 - before fourth, see if anyone in another group that is a second choice has a second choice that is not over capacity
        self.AdjustGroups(4) #to fourth
        self.ComplexAdjustGroups(3) #3+3
        self.AdjustGroups(5) #to fifth

    def NewEliminatedGroup(self):
        groups = [group for group in self.groups if group not in self.eliminatedGroups]
        lengths = [self.getLengthGroupName(group) for group in groups]
        if min(lengths) > self.minCapacity: return None
        group = groups[lengths.index(min(lengths))]
        #[group for group in self.groups if self.getLengthGroupName(group) < self.minCapacity]
        return group
    def AdjustUnderCapacity(self): #put undercapacity choice in next best group for them even if over capacity
        for groupName in self.eliminatedGroups:
            for person,currentRank in self.groups[groupName]:
                moved = False
                for newRank in range(1,6): 
                    if newRank == currentRank: continue
                    
                    newChoice = self.getPersonsRank(person,newRank)
                    if newChoice in self.eliminatedGroups: continue
                    
                    self.removePerson(person)
                    self.addPersonToGroup(newChoice,person,newRank)
                    moved = True
                    break
                if not moved: 
                    self.flagged.append(person)
                
                    
                        
                    
   
        
    def Move(self,groupName,possibleRanks):
        group = self.getGroup(groupName)
        for newRank in possibleRanks:
            for person,currentRank in group:
                if newRank == currentRank: continue
                newChoice = self.getPersonsRank(person,newRank)
                if newChoice in self.eliminatedGroups: continue
                if self.getLengthGroupName(newChoice)<self.maxCapacity:

                    self.removePerson(person)
                    self.addPersonToGroup(newChoice,person,newRank)
                
                if self.getLengthGroupName(groupName)<=self.maxCapacity:
                    break
                    
    def removeRankFromRanks(self,currentRank,ranks):
        return [rank for rank in ranks if rank != currentRank]
    
    def AdjustGroups(self,newRank):
        possibleRanks = range(1,newRank+1)
        self.OverCapacityGroups = self.overCapacity()
        for groupName in self.OverCapacityGroups:
            self.Move(groupName,possibleRanks)
            
   
    def ComplexAdjustGroups(self,newRank):
        possibleRanks = range(1,newRank+1)
        self.OverCapacityGroups = self.overCapacity()
        options = self.OtherPreferredChoices(possibleRanks)
        self.MakeRoomForSomeoneElse(options,possibleRanks)
    
    def OtherPreferredChoices(self, possibleRanks): #find prefered choices in this group in top ranks if they can't be in their own group
        options = {}
        for groupName in self.OverCapacityGroups:
            for person, currentRank in self.groups[groupName]:
                choices = self.getChoices(person)
                rankOptions = self.removeRankFromRanks(currentRank,possibleRanks)
                for newRank in rankOptions:

                    newChoice = self.getRank(choices,newRank)
                    if newChoice in self.eliminatedGroups: continue

                    if self.getLengthGroupName(newChoice)<=self.maxCapacity:#the other groups need to get rid of people for themselves
                        options[newChoice] = options.get(newChoice,[])+[[person,newRank]] #create a dictionary with the people who want this new group and the rank of how much they want it

        return options
    
    def findReplacement(self,options,newGroup):
        options = {person:rank for person, rank in options[newGroup]}
        newPerson = sorted(options,key=options.get)[0]
        newRank = options[newPerson]
        return newPerson,newRank
    
    def MakeRoomForSomeoneElse(self,options,possibleRanks):
        for group in options:
            for newRank in possibleRanks:
                for person,currentRank in self.groups[group]:
                    if newRank == currentRank: continue
                    newChoice = self.getPersonsRank(person,newRank)
                    if newChoice in self.eliminatedGroups: continue
                    if self.getLengthGroupName(newChoice)<self.maxCapacity:
                        
                        newPerson,newPersonRank = self.findReplacement(options,group)
                        self.removePerson(newPerson)
                        self.addPersonToGroup(group,newPerson,newPersonRank)
                        
                        self.removePersonFromGroup(group,person)
                        self.addPersonToGroup(newChoice,person,newRank)
    def Evaluate(self):
        print(self.groups)
        ranks = {}
        for group in self.groups:
            for person,rank in self.groups[group]:
                ranks[rank] = ranks.get(rank,0)+1
        for x in range(1,6):
            print("# of people with rank %s: %s" % (x,ranks.get(x,0)))
                        
    

                

In [20]:
obj = Scheduling()

Eliminated Groups []
GroupName: 1 # of people 12
GroupName: 2 # of people 3
GroupName: 3 # of people 8
GroupName: 4 # of people 7
GroupName: 5 # of people 0
GroupName: 6 # of people 5
GroupName: 7 # of people 11
GroupName: 8 # of people 8
GroupName: 9 # of people 12
GroupName: 10 # of people 8
GroupName: 11 # of people 5
GroupName: 12 # of people 7
GroupName: 13 # of people 5
GroupName: 14 # of people 6
GroupName: 15 # of people 3



Eliminated Groups [5]
GroupName: 1 # of people 12
GroupName: 2 # of people 3
GroupName: 3 # of people 8
GroupName: 4 # of people 7
GroupName: 5 # of people 0
GroupName: 6 # of people 5
GroupName: 7 # of people 11
GroupName: 8 # of people 8
GroupName: 9 # of people 12
GroupName: 10 # of people 8
GroupName: 11 # of people 5
GroupName: 12 # of people 7
GroupName: 13 # of people 5
GroupName: 14 # of people 6
GroupName: 15 # of people 3



Eliminated Groups [5, 2]
GroupName: 1 # of people 12
GroupName: 2 # of people 0
GroupName: 3 # of people 9
GroupName: 4 # o

In [116]:
[1,2,2].pop(1)

2

In [ ]:
k = [1]
for x in k:
    k.append(2)